### import 및 전역변수
- 컷오프일 pkl 폴더의 파일 읽기 : 채무자, 계좌, 보증인, 개회, 신복
- 매각일 추가 파일 다운 및 읽기 : 매각일기준계좌, 메모, 법조치, 입금

In [1]:
import pandas as pd
from tqdm import tqdm
import re, os
from os.path import join

account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
memo_dtype = {'채무자키' : str, '메모키':str}
event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}
deposit_dtype = {'채무자키' : str, '입금키':str, '계좌키':str, '입금계좌번호':str, '계좌번호':str, '타입금키':str, '타채무자키':str, '보증인키':str}

In [2]:
################################################################
company = "솔림"
cutDate = "240528"
sellDate = "240627"
sell_ext = ".xlsx" # 매각일 전산자료. 처음 받았다면 xlsx
매각사 = "애플"
path_save_base = r"D:\3.자산\POOL별 관리자산\솔림\POOL 12 번자료 (스마트저축-01, 우리금융캐피탈)\매각\240627_애플_조정,무담보"
path_매각일파일들 = join(path_save_base, "전산 rawdata") # 매각 폴더 안에 전산로우데이터 다운, 메모-계좌-법조치-입금
path_매각키리스트 = join(path_save_base, "240627_P12_애플_키리스트"+".xlsx") # 채무자키, 계좌키 만
################################################################

path_savefile = join(path_save_base, "솔림_전산자료_"+매각사+"_"+cutDate+".xlsx")
cutDate_to_input = "20"+cutDate[:2]+"-"+cutDate[2:4]+"-"+cutDate[4:]
sellDate_to_input = "20"+sellDate[:2]+"-"+sellDate[2:4]+"-"+sellDate[4:]

### 파일 읽기

In [ ]:
# 파일읽기 # 매각일 자료도 일단은 dataset/pkl폴더에 저장하고 나중에 옮기든지 하자.
cut_ext = ".pkl" # 자산정리하면서 pkl 만들었으니
########################################################################################################################
cutFiles = [name + cut_ext for name in ["채무자조회새창_20240528_1722", "계좌조회새창_20240528_1725", "보증인새창_20240528_1729", \
    "개인회생새창_20240529_0950_진행중_계좌키분리_기준데이터", "240528_신용회복_기준데이터_약식병합_솔림"]]
                                                            ###### 신복, 개회 기준데이터로 ###### 그래야 자산정리와 같고, 날짜랑도 맞는다.
sellFiles = ["계좌조회새창_20240628_0952", "메모조회새창_20240628_0936", "법조치조회새창_20240628_0937", "입금조회새창_20240628_0954"]
########################################################################################################################

# 자산확정일 기준 자료들
cut_dir = join(r"D:\3.자산\전산 dataset", company, cutDate, "pkl")
debt = pd.read_pickle(join(cut_dir, cutFiles[0]))
cut_accout = pd.read_pickle(join(cut_dir, cutFiles[1]))
grt = pd.read_pickle(join(cut_dir, cutFiles[2]))
rehabilitation = pd.read_pickle(join(cut_dir, cutFiles[3]))
credit = pd.read_pickle(join(cut_dir, cutFiles[4]))

# 매각일 기준 자료들
if sell_ext == ".xlsx" : 
    sell_account = pd.read_excel(join(path_매각일파일들, sellFiles[0]+sell_ext), dtype=account_dtype ).fillna("")
    memo = pd.read_excel(join(path_매각일파일들, sellFiles[1]+sell_ext), dtype=memo_dtype ).fillna("")
    event = pd.read_excel(join(path_매각일파일들, sellFiles[2]+sell_ext), dtype=event_dtype ).fillna("")
    deposit = pd.read_excel(join(path_매각일파일들, sellFiles[3]+sell_ext), dtype=deposit_dtype ).fillna("")
else : # pkl
    sell_account = pd.read_pickle(join(path_매각일파일들, sellFiles[0]+sell_ext))
    memo = pd.read_pickle(join(path_매각일파일들, sellFiles[1]+sell_ext))
    event = pd.read_pickle(join(path_매각일파일들, sellFiles[2]+sell_ext))
    deposit = pd.read_pickle(join(path_매각일파일들, sellFiles[3]+sell_ext))

# 신복기준데이터의 No.열 없애기
credit = credit.drop(['순번'], axis=1) # No. > 순번

# 매각리스트
keys = pd.read_excel(path_매각키리스트, dtype=str).fillna("")

In [6]:
# # 매각일 자료 pkl 저장 -----------
# sell_account.to_pickle(join(path_매각일파일들, sellFiles[0]+".pkl"))
# memo.to_pickle(join(path_매각일파일들, sellFiles[1]+".pkl"))
# event.to_pickle(join(path_매각일파일들, sellFiles[2]+".pkl"))
# deposit.to_pickle(join(path_매각일파일들, sellFiles[3]+".pkl"))

### 데이터 추출

In [7]:
# merge(df-df)를 위해 키 중복값 제거
debtkeys = keys[["채무자키"]].drop_duplicates('채무자키') # df리턴
accountkeys = keys[["계좌키"]].drop_duplicates('계좌키') # df리턴 # 중복값 없겠지만 혹시 모르니
# 채무자키와 새채무자키
new_debtkeys = debt[["채무자키","관리자비고"]].merge(debtkeys, how="right").rename(columns = {"관리자비고":"새채무자키"})
print(len(debtkeys), len(accountkeys), len(new_debtkeys))

1528 1528 1528


In [8]:
# 법조치 새채무자키 추가하기 (new_debekeys의 채무자키에 중복값이 없으므로 숫자변화 없다)
print(len(event))
event = event.merge(new_debtkeys, on='채무자키', how='left')
print(len(event))

102
102


In [9]:
# 추출하기 : 법조치는 새채무자키로 추출
# 자산확정일 : 채무자, 계좌, 보증인, 개인회생, 신용회복
# 매각일 : 계좌, 입금(계좌조회,계좌별표시,기간제한x), 메모, 법조치(새채무자키)
# 채무자 : 채무자키
debt = debt.merge(debtkeys, how='inner')
# 계좌_확정일 : 계좌키 - 계좌키 누락/일부 매각 없어야
cut_accout = cut_accout.merge(accountkeys, how='inner')
# 보증인 : 계좌키
grt = grt.merge(accountkeys, how='inner') # 보증인
# 개인회생 : 계좌키
rehabilitation = rehabilitation.merge(accountkeys, how='inner')
# 신용회복 : 계좌키
credit = credit.merge(accountkeys, how='inner')
# 법조치, 채무자키 기준 vs 새채무자키 기준 - 계좌키는 제대로 등록이 되어있지 않으므로 사용할 수 없다.
event_채무자키 = event.merge(debtkeys, how='inner', on='채무자키')
event = event.merge(new_debtkeys[['새채무자키']], how='inner', on='새채무자키')
print(f"법조치 - 채무자키로 법조치 추출 : {len(event_채무자키)}, 새채무자키로 법조치 추출 : {len(event)}")
# 메모 : 채무자키
memo = memo.merge(debtkeys, how='inner')
# 계좌_매각일
sell_account = sell_account.merge(accountkeys, how='inner')
# 입금 : 계좌키
deposit = deposit.merge(accountkeys, how='inner')

# 법조치 새채무자키든 채무자키든 결과차이 없다면 새채무자키 열 삭제 
if len(event_채무자키) == len(event) : 
    event.drop(['새채무자키'], axis=1)

법조치 - 채무자키로 법조치 추출 : 102, 새채무자키로 법조치 추출 : 109


### 칼럼 편집

In [10]:
dfs = [debt, cut_accout, grt, rehabilitation, credit, event, memo, deposit, sell_account]
df_names = ["채무자", "계좌", "보증인", "개인회생", "신용회복", "법조치_"+sellDate, "memo_"+sellDate, "deposit_"+sellDate, "계좌_"+sellDate]

# # 문제되는 칼럼 살피기
# x = set()
# y = {"debt" : [], "cut_accout" : [], "grt" : [], "rehabilitation" : [], "credit" : [], "event" : [], "memo" : [], "deposit" : [], "sell_account" : []}
# for i, df in enumerate(dfs):
#     for col in df.columns :
#         if re.search('이자기준일', col) :
#             y.get(df_names[i]).append(col)
#             x.add(col)
# print(x)
# for k,v in y.items() : 
#     print(k, ":", v)

In [11]:
# 컬럼 내용 편집
# 초기화할 열 목록
columns_to_reset = ['담당자','담당자키','담당핸드폰','송무담당', '담당입금구분', '담당문자전송', '담당연락처', '전담당자', '담당변경일', '지점','지역지점', "채무상태", 
     '부서', '팀', '등록파일', '등록유저', '수정파일', '법조치파일','등록유저', '수정유저','OPB', '담보제공처', '담보제공구분', '채권매각일', '채권매각대금', '채권매각처', '채권환매대금','채권환매일',
     '채권손실일', '채권손실금액', '계약일', '입금은행', '상대방입금은행', '입금계좌번호', '입금순서'] # OPB 추가:우리 매입당시 OPB 알려줄 필요없으니 알아서 계산해서 쓰라고
columns_to_input = ["매입일", "컷오프일", "최초이자기준일"]

# 모든 데이터프레임에 대해 열 초기화
for df in dfs:
    for col in columns_to_reset: # 삭제
        if col in df.columns:
            df[col] = ""
    for ind, col in enumerate(columns_to_input): # 컷오프일 or 매입일 작성
        if col in df.columns:
            if ind==0 : df[col] = sellDate_to_input
            else : df[col] = cutDate_to_input

In [12]:
# 메모시트 등록일자 편집 및 매각메모 삭제
############################################################
삭제할메모내용 = "23.08.31 매각(애플) 02-3474-0522"
############################################################
# 공지 기호 삭제
memo["등록일자"] = memo.등록일자.str.replace("★ | ★", "", regex=True)
# 매각 메모 삭제
before = len(memo)
memo = memo[~(memo.메모내용 == 삭제할메모내용)]
memo.reset_index(drop=True)
after = len(memo)
print(f"삭제한 메모수 : {before} - {after} = {before-after}, 매각한 채무자키 수 : {len(debtkeys)}")
# 등록일자순 정렬
memo = memo.sort_values(['등록일자'], ascending=False)
# 메모색상, 공지여부
memo['메모색상'] = "검정"
memo['공지여부'] = "N"

삭제한 메모수 : 4801 - 4801 = 0, 매각한 채무자키 수 : 1528


### 저장

In [13]:
# 엑셀 저장 (파일이 오류나네.. 근데 내용은 문제 없어서, 복구 후 다른이름으로 저장하면 괜찮았다.)
# dfs에 변경된 내용 반영 (drop과 sort가 반영이 안 되네..)
dfs = [debt, cut_accout, grt, rehabilitation, credit, event, memo, deposit, sell_account]
df_names = ["채무자", "계좌", "보증인", "개인회생", "신용회복", "법조치_"+sellDate, "메모_"+sellDate, "입금_"+sellDate, "계좌_"+sellDate]
dict_sheet_df = dict(zip(df_names, dfs))

with pd.ExcelWriter(path_savefile, engine='openpyxl', mode = 'w') as writer:
    for sheetname, df_name in dict_sheet_df.items():
        df_name.to_excel(writer, sheet_name = sheetname, index=False)

In [ ]:
# readme - 다음을 복사
readme = """
시트명에 날짜가 없는 것은 컷오프일 기준의 전산데이터입니다.
시트명에 날짜가 붙은 것은 매각일 기준의 전산데이터입니다.

삭제정보 : 불필요한 정보(솔림내부정보 : 담당자, 입금계좌 등), 재작성 해야하는 정보(상태,OPB 등)는 삭제하였습니다.
수정정보 : 매입일, 컷오프일, 최초이자기준일은 금번 계약 내용으로 수정하여 두었습니다.
유지정보(등록시 수정필요) : 매각사, 각종 키값 및 타키값(타채무자키 등)은 참고가 필요할 수 있어 일단 솔림 전산값 그대로 두었으니 전산 등록시 수정하시기 바랍니다.
"""
#    법조치의 새채무자키 - 매각대상에 없는 채무자키가 들어있을 수 있습니다. 이는 채무자키는 다르지만, ....